In [1]:
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import json
import random
import os
import glob
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def check_units(y_true, y_pred):
    if y_pred.shape[1] != 1:
      y_pred = y_pred[:,1:2]
      y_true = y_true[:,1:2]
    return y_true, y_pred

def precision(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    y_true, y_pred = check_units(y_true, y_pred)
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def matthews_correlation(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [3]:
model = None
model = load_model("vgg_sterioid.h5", custom_objects={'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc, 'matthews_correlation': matthews_correlation})
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [4]:
base_path = "/"
csv_path = "csv/"
cell_images_data = "cell_images/"

In [5]:
patient_records = None
with open('patient_records.json', 'r') as fp:
    patient_records = json.load(fp)
    
patient_labels = np.load("patient_labels.npy")
patient_labels.shape

(200,)

In [6]:
patient_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int64)

In [7]:
tot = 0
for name, paths in patient_records.items():
    print(name, len(paths))
    tot+= len(paths)
print(tot)

C100P61ThinF 125
C101P62ThinF 385
C102P63ThinF 103
C103P64ThinF 107
C104P65ThinF 82
C105P66ThinF 85
C106P67ThinF 79
C107P68ThinF 132
C108P69ThinF 89
C109P70ThinF 87
C110P71ThinF 93
C111P72ThinF 88
C112P73ThinF 79
C113P74ThinF 127
C114P75ThinF 81
C115P76ThinF 78
C116P77ThinF 482
C117P78ThinF 148
C118P79ThinF 353
C119P80ThinF 182
C120P81ThinF 164
C121P82ThinF 84
C122P83ThinF 75
C123P84ThinF 131
C124P85ThinF 114
C125P86ThinF 83
C126P87ThinF 120
C127P88ThinF 82
C128P89ThinF 206
C129P90ThinF 386
C12NThinF 68
C130P91ThinF 185
C131P92ThinF 119
C132P93ThinF 639
C133P94ThinF 347
C134P95ThinF 98
C135P96ThinF 76
C136P97ThinF 397
C137P98ThinF 408
C138P99ThinF 74
C139P100ThinF 86
C13NThinF 69
C140P101ThinF 167
C141P102ThinF 92
C142P103ThinF 85
C143P104ThinF 81
C144P105ThinF 148
C145P106ThinF 107
C146P107ThinF 111
C147P108ThinF 79
C148P109ThinF 93
C149P110ThinF 121
C150P111ThinF 68
C151P112ThinF 73
C152P113ThinF 80
C153P114ThinF 72
C154P115ThinF 91
C155P116ThinF 69
C156P117ThinF 99
C157P118ThinF 107

In [8]:
# key- patient name
# value - numpy array of batch of images
patient_batch_images = dict()

In [9]:
# helper to get images
def get_data(path, resize_dim=200):
    #print(path)
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    #img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    #img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    
    img_rz=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA)
    return img_rz

In [10]:
total = 0
for name, paths in patient_records.items():
    
    # exit after a few iterations
    #if len(paths) == 89:
        #break
        
    batch_of_images = None
    batch_of_images_paths = []
    
    for p in paths:
        
        img_path = p
        if "'" in img_path:
            #print("' exists")
            img_path = img_path.strip("'")
        
        # path without '    
        #print(img_path)
        
        if img_path in os.listdir(cell_images_data+"Parasitized"):
            batch_of_images_paths.append(cell_images_data+"Parasitized/"+img_path)
        else:
            batch_of_images_paths.append(cell_images_data+"Uninfected/"+img_path)
            
    total+= len(batch_of_images_paths)
    batch_of_images =np.array([get_data(path) for path in batch_of_images_paths])
    print(len(batch_of_images_paths), batch_of_images.shape)
    
    patient_batch_images[name] = batch_of_images
    batch_of_images = None
    batch_of_images_paths = []
    
    

print(total)
print(len(patient_batch_images.keys()))

125 (125, 200, 200, 3)
385 (385, 200, 200, 3)
103 (103, 200, 200, 3)
107 (107, 200, 200, 3)
82 (82, 200, 200, 3)
85 (85, 200, 200, 3)
79 (79, 200, 200, 3)
132 (132, 200, 200, 3)
89 (89, 200, 200, 3)
87 (87, 200, 200, 3)
93 (93, 200, 200, 3)
88 (88, 200, 200, 3)
79 (79, 200, 200, 3)
127 (127, 200, 200, 3)
81 (81, 200, 200, 3)
78 (78, 200, 200, 3)
482 (482, 200, 200, 3)
148 (148, 200, 200, 3)
353 (353, 200, 200, 3)
182 (182, 200, 200, 3)
164 (164, 200, 200, 3)
84 (84, 200, 200, 3)
75 (75, 200, 200, 3)
131 (131, 200, 200, 3)
114 (114, 200, 200, 3)
83 (83, 200, 200, 3)
120 (120, 200, 200, 3)
82 (82, 200, 200, 3)
206 (206, 200, 200, 3)
386 (386, 200, 200, 3)
68 (68, 200, 200, 3)
185 (185, 200, 200, 3)
119 (119, 200, 200, 3)
639 (639, 200, 200, 3)
347 (347, 200, 200, 3)
98 (98, 200, 200, 3)
76 (76, 200, 200, 3)
397 (397, 200, 200, 3)
408 (408, 200, 200, 3)
74 (74, 200, 200, 3)
86 (86, 200, 200, 3)
69 (69, 200, 200, 3)
167 (167, 200, 200, 3)
92 (92, 200, 200, 3)
85 (85, 200, 200, 3)
81 (81, 2

In [11]:
#np.save("data_sub.npy", patient_batch_images)

In [12]:
for name, batch in patient_batch_images.items():
    print(name, batch.shape)

C100P61ThinF (125, 200, 200, 3)
C101P62ThinF (385, 200, 200, 3)
C102P63ThinF (103, 200, 200, 3)
C103P64ThinF (107, 200, 200, 3)
C104P65ThinF (82, 200, 200, 3)
C105P66ThinF (85, 200, 200, 3)
C106P67ThinF (79, 200, 200, 3)
C107P68ThinF (132, 200, 200, 3)
C108P69ThinF (89, 200, 200, 3)
C109P70ThinF (87, 200, 200, 3)
C110P71ThinF (93, 200, 200, 3)
C111P72ThinF (88, 200, 200, 3)
C112P73ThinF (79, 200, 200, 3)
C113P74ThinF (127, 200, 200, 3)
C114P75ThinF (81, 200, 200, 3)
C115P76ThinF (78, 200, 200, 3)
C116P77ThinF (482, 200, 200, 3)
C117P78ThinF (148, 200, 200, 3)
C118P79ThinF (353, 200, 200, 3)
C119P80ThinF (182, 200, 200, 3)
C120P81ThinF (164, 200, 200, 3)
C121P82ThinF (84, 200, 200, 3)
C122P83ThinF (75, 200, 200, 3)
C123P84ThinF (131, 200, 200, 3)
C124P85ThinF (114, 200, 200, 3)
C125P86ThinF (83, 200, 200, 3)
C126P87ThinF (120, 200, 200, 3)
C127P88ThinF (82, 200, 200, 3)
C128P89ThinF (206, 200, 200, 3)
C129P90ThinF (386, 200, 200, 3)
C12NThinF (68, 200, 200, 3)
C130P91ThinF (185, 200, 20

In [13]:
results = []

for name, batch in patient_batch_images.items():
    print(name, batch.shape)
    
    y_preds = model.predict(batch, verbose=1)
    y_pred_flat = np.argmax(y_preds, axis=1)
    if 1 in y_pred_flat:
        results.append(1)
    else:
        results.append(0)
    y_preds = None
    y_pred_flat = None

print(len(results))
print("Done!")

C100P61ThinF (125, 200, 200, 3)
125/125 [==============================] - 4s 34ms/step
C101P62ThinF (385, 200, 200, 3)
385/385 [==============================] - 2s 6ms/step
C102P63ThinF (103, 200, 200, 3)
103/103 [==============================] - 1s 11ms/step
C103P64ThinF (107, 200, 200, 3)
107/107 [==============================] - 1s 12ms/step
C104P65ThinF (82, 200, 200, 3)
82/82 [==============================] - 1s 17ms/step
C105P66ThinF (85, 200, 200, 3)
85/85 [==============================] - 2s 18ms/step
C106P67ThinF (79, 200, 200, 3)
79/79 [==============================] - 1s 16ms/step
C107P68ThinF (132, 200, 200, 3)
132/132 [==============================] - 1s 9ms/step
C108P69ThinF (89, 200, 200, 3)
89/89 [==============================] - 2s 19ms/step
C109P70ThinF (87, 200, 200, 3)
87/87 [==============================] - 2s 18ms/step
C110P71ThinF (93, 200, 200, 3)
93/93 [==============================] - 0s 5ms/step
C111P72ThinF (88, 200, 200, 3)
88/88 [===============

C203ThinF (69, 200, 200, 3)
69/69 [==============================] - 0s 5ms/step
C204ThinF (67, 200, 200, 3)
67/67 [==============================] - 0s 5ms/step
C205ThinF (69, 200, 200, 3)
69/69 [==============================] - 0s 5ms/step
C206ThinF (71, 200, 200, 3)
71/71 [==============================] - 0s 5ms/step
C207ThinF (70, 200, 200, 3)
70/70 [==============================] - 0s 5ms/step
C208ThinF (69, 200, 200, 3)
69/69 [==============================] - 0s 5ms/step
C209ThinF (71, 200, 200, 3)
71/71 [==============================] - 0s 5ms/step
C210ThinF (68, 200, 200, 3)
68/68 [==============================] - 0s 5ms/step
C211ThinF (69, 200, 200, 3)
69/69 [==============================] - 0s 5ms/step
C212ThinF (68, 200, 200, 3)
68/68 [==============================] - 0s 5ms/step
C213ThinF (69, 200, 200, 3)
69/69 [==============================] - 0s 5ms/step
C214ThinF (67, 200, 200, 3)
67/67 [==============================] - 0s 5ms/step
C215ThinF (69, 200, 200, 3)


333/333 [==============================] - 2s 5ms/step
C92P53ThinF (93, 200, 200, 3)
93/93 [==============================] - 0s 5ms/step
C93P54ThinF (153, 200, 200, 3)
153/153 [==============================] - 1s 5ms/step
C94P55ThinF (79, 200, 200, 3)
79/79 [==============================] - 0s 5ms/step
C95P56ThinF (92, 200, 200, 3)
92/92 [==============================] - 0s 5ms/step
C96P57ThinF (86, 200, 200, 3)
86/86 [==============================] - 0s 6ms/step
C97P58ThinF (310, 200, 200, 3)
310/310 [==============================] - 2s 5ms/step
C98P59ThinF (108, 200, 200, 3)
108/108 [==============================] - 1s 5ms/step
200
Done!


In [14]:
results

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [15]:
results = np.array(results)
len(results)

200

In [16]:
type(patient_labels), type(results)

(numpy.ndarray, numpy.ndarray)

In [17]:
len(patient_labels), len(results)

(200, 200)

In [20]:
accuracy = np.mean((patient_labels==results))
accuracy

0.93

In [18]:
auc_val = auc(patient_labels, results)
auc_val

<tf.Tensor 'auc/update_op:0' shape=() dtype=float32>

In [20]:
import numpy as np
from sklearn.metrics import roc_auc_score
#>>> y_true = np.array([0, 0, 1, 1])
#y_scores = np.array([0.1, 0.4, 0.35, 0.8])
roc_auc_score(patient_labels, results)

0.8653846153846154

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(patient_labels, results)

0.93

In [22]:
np.save("results.npy", results)